# Uploading our stuff

    First, lets import our python toolbox and the data

In [1]:
import pandas as pd
import psycopg2

In [2]:
dfip = pd.read_excel('InPerfectOutput.xlsx')
print(dfip.head())

   Ser     Category             SubCategory         SubSubCategory  \
0    5  Agriculture  Agricultural Equipment  Aquaculture Equipment   
1    6  Agriculture  Agricultural Equipment  Aquaculture Equipment   
2    7  Agriculture  Agricultural Equipment  Aquaculture Equipment   
3    8  Agriculture  Agricultural Equipment  Aquaculture Equipment   
4    9  Agriculture  Agricultural Equipment  Aquaculture Equipment   

             SubSubSubCategory  
0             Aquaculture Trap  
1                Fishing Float  
2                  Fishing Net  
3                 Fishing Rope  
4  Other Aquaculture Equipment  


In [3]:
dfp = pd.read_excel('PerfectOutput.xlsx')
print(dfp.head())

   Ser     Category             SubCategory            SubSubCategory  \
0    3  Agriculture  Agricultural Equipment  Agricultural Greenhouses   
1    4  Agriculture  Agricultural Equipment     Aquaculture Equipment   
2    5  Agriculture  Agricultural Equipment     Aquaculture Equipment   
3    6  Agriculture  Agricultural Equipment     Aquaculture Equipment   
4    7  Agriculture  Agricultural Equipment     Aquaculture Equipment   

  SubSubSubCategory  
0               NaN  
1               NaN  
2  Aquaculture Trap  
3     Fishing Float  
4       Fishing Net  


In [4]:
dfSlim = pd.read_excel('slimTree.xlsx')
print(dfSlim.head())

                   category               subcategory
0               Agriculture    Agricultural Equipment
1    Agricultural Equipment  Agricultural Greenhouses
2  Agricultural Greenhouses                       NaN
3               Agriculture    Agricultural Equipment
4    Agricultural Equipment     Aquaculture Equipment


In [10]:
con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

In [11]:
# select * from "Categories" c;
# insert into "Categories" (category) values ('food');

cursor.execute("""select * from "Categories" c""")
cursor.execute("""insert into "Categories" (category) values ('food but with py again this time')""")


    Read from DB to test

In [12]:
cursor.execute("""select * from "Categories" c""")

print(cursor.fetchall())

[(563, 'food99'), (564, 'food but with py again this time')]


    Due to a few issues, lets unit test this again

In [13]:
cursor.execute("""select * from "Categories" c""")
cursor.execute("""insert into "Categories" (category) values ('unique code which will be in table')""")

cursor.execute("""select * from "Categories" c""")
print(cursor.fetchall())

[(563, 'food99'), (564, 'food but with py again this time'), (565, 'unique code which will be in table')]


    Looks like reading & writing works, now lets try deleting

In [15]:
cursor.execute("""select * from "Categories" c""")
category_id = "565"; category_name = 'unique code which will be in table'
cursor.execute("""DELETE FROM "Categories" WHERE category_id = """ + category_id)# + """ OR category = """ + category_name)# type: ignore #, (category_id, category_name))

cursor.execute("""select * from "Categories" c""")
print(cursor.fetchall())

[(563, 'food99'), (564, 'food but with py again this time')]




    Now lets make a similar version for the categroy tree:

In [63]:
# cursor.execute("""select * from "Category_Tree" ct""")
# cursor.execute("""insert into "Category_Tree" (category_id, sub_category_id) values ("catid", "subcatid")""")

# cursor.execute("""select * from "Categories" c""")
# print(cursor.fetchall())

    this would work to pass a string, but Category_Tree's category & sub category parameters both take integers, so lets get encoding

    First lets pass an old data set from pdf processing, the following cell is recycled code

In [5]:
df1 = pd.read_excel('document.xlsx')
midpoint = len(df1) // 2 # type: ignore
df2a = df1.iloc[:midpoint].reset_index(drop=True)
df2b = df1.iloc[midpoint:].reset_index(drop=True)
df2b = df2b.drop(['Category', 'SubCategory'], axis=1) # drop na collumns
df2b = df2b.iloc[1:] # remove unneccesary row with innacurate names
df2b.columns = ['SubSubCategory', 'SubSubSubCategory']
df2a = df2a.drop(['Unnamed: 3'], axis=1) # drop
df2b['Ser'] = range(1, len(df2b) + 1) # type: ignore
df3 = pd.merge(df2a, df2b, on='Ser', suffixes=('_a', '_b'))

print(df3)

       Ser     Category             SubCategory            SubSubCategory  \
0        1  Agriculture                     NaN                       NaN   
1        2          NaN  Agricultural Equipment                       NaN   
2        3          NaN                     NaN  Agricultural Greenhouses   
3        4          NaN                     NaN     Aquaculture Equipment   
4        5          NaN                     NaN                       NaN   
...    ...          ...                     ...                       ...   
4591  4592          NaN                     NaN            Truck Steering   
4592  4593          NaN                     NaN          Truck Suspension   
4593  4594          NaN                     NaN                Truck Tire   
4594  4595          NaN                     NaN        Truck Transmission   
4595  4596          NaN                     NaN               Truck Wheel   

     SubSubSubCategory  
0                  NaN  
1                  NaN  


    Now lets try making a map from this

In [6]:
df = pd.DataFrame(df3)

def combine_non_na(row):
    return ' '.join([str(val) for val in row if val != 'na' and not isinstance(val, (int, float))])

df['combined'] = df.apply(combine_non_na, axis=1)

df = df[['Ser', 'combined']]

print(df)

       Ser                  combined
0        1               Agriculture
1        2    Agricultural Equipment
2        3  Agricultural Greenhouses
3        4     Aquaculture Equipment
4        5          Aquaculture Trap
...    ...                       ...
4591  4592            Truck Steering
4592  4593          Truck Suspension
4593  4594                Truck Tire
4594  4595        Truck Transmission
4595  4596               Truck Wheel

[4596 rows x 2 columns]


    Lets save this for safe keeping

In [18]:
df.to_excel('map.xlsx', index=False)

    Now its DB time, we need to put this map into 1 of our tables, & use the map for our other table

    Lets start out by uploading the map we just made

    First lets make sure the db is empty

In [9]:
cursor.execute("""select * from "Categories" c""")
print(cursor.fetchall())

# cursor.execute("""select * from "Categories" c""")
# cursor.execute("""insert into "Categories" (category) values ('food but with py again this time')""")

# cursor.execute("""select * from "Categories" c""")
# cursor.execute("""drop table "Categories" """)

[]


In [98]:

# cursor.execute("""select * from "Categories" c""")
# cursor.execute("""insert into "Categories" (category) values ('food but with py again this time')""")

# cursor.execute("""select * from "Categories" c""")
# print(cursor.fetchall())

    Now lets touch the DB

In [8]:
con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

In [28]:
def get_category_subcategory(df, index):
    num = df.loc[index, 'Ser']
    name = df.loc[index, 'combined']
    return num, name

for i in range(1000): # first 1000
    num, name = get_category_subcategory(df, i); num = str(num)
    
    cursor.execute("""select * from "Categories" c""")
    cursor.execute("""insert into "Categories" (category_id, category) values (""" + num + """, '""" + name + """')""") # type: ignore

# con.commit()

UniqueViolation: duplicate key value violates unique constraint "accounts_category_key"
DETAIL:  Key (category)=() already exists.


    A unique violation? Exception time

In [29]:
listOfCollisions = []

def get_category_subcategory(df, index):
    num = df.loc[index, 'Ser']
    name = df.loc[index, 'combined']
    return num, name

for i in range(len(df)): # type: ignore
# for i in range(10): # type: ignore
    num, name = get_category_subcategory(df, i); num = str(num)
    
    try:
        
        try:    
            cursor.execute("""select * from "Categories" c""")
            cursor.execute("""insert into "Categories" (category) values ('""" + name + """')""") # type: ignore
        except Exception as e:
            cursor.execute("""insert into "Categories" (category) values ('""" + name + """')""") # type: ignore
        
        
    except psycopg2.errors.UniqueViolation as e:
        listOfCollisions.append(i)

con.commit() # add this to make it the real thing

print(listOfCollisions.__sizeof__())

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


    Another issue? Lets try going a couple hundred terms at a time, & Im gonna clean up the code while we're at it

In [30]:

def process_categories(df, cursor, con, start, end):
    listOfCollisions = []

    def get_category_subcategory(df, index):
        num = df.loc[index, 'Ser']
        name = df.loc[index, 'combined']
        return num, name

    for i in range(start, end):
        num, name = get_category_subcategory(df, i)
        num = str(num)

        try:
            cursor.execute("""INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""", (num, name))
        except psycopg2.errors.UniqueViolation as e:
            listOfCollisions.append(i)
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    con.commit()

    print(len(listOfCollisions)); return listOfCollisions

In [34]:
collisions = process_categories(df, cursor, con, 0, 100)


0


    16 seconds for the first 100 entries? No collisions? Not bad
    Now this should take ~33 seconds

In [71]:
collisions = process_categories(df, cursor, con, 101, 300)

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

A

    Hm, weird output there - but worst of all no new data appeared on DBeaver - lets try:

In [35]:
collisions = process_categories(df, cursor, con, 101, 150)

0


    Success! lets try:

In [36]:
collisions = process_categories(df, cursor, con, 151, 200)

0


    Now lets:

In [37]:
collisions = process_categories(df, cursor, con, 201, 250)

0


In [38]:
collisions = process_categories(df, cursor, con, 251, 300)

0


In [39]:
collisions = process_categories(df, cursor, con, 301, 350)

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

A

    Theres the issue, one of the early 300's is messing this up, lets wipe the db & try a smaller interval

In [42]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

collisions = process_categories(df, cursor, con, 0, 300)

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block

An unexpected error occurred: current transaction is aborted, commands ignored until end of transaction block



KeyboardInterrupt: 

    // Cell aborted

    One possible explanation: lets move around the commit command, & while here optimize the code

In [52]:

def get_category_subcategory(df, index): # define this once cause micro optimizing is fun
    num = df.loc[index, 'Ser']
    name = df.loc[index, 'combined']
    return name, num

def process_categories(df, cursor, con, start, end):
    listOfCollisions = []

    for i in range(start, end):
        name, num = get_category_subcategory(df, i); num = str(num)

        try:
            cursor.execute("""INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""", (num, name))
        except psycopg2.errors.UniqueViolation as e:
            listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
        
        con.commit() # we moved this

    print(len(listOfCollisions)); return listOfCollisions

In [53]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

collisions = process_categories(df, cursor, con, 0, 100)
print("1st 100 done")
collisions = process_categories(df, cursor, con, 100, 200)
print("2nd 100 done")
collisions = process_categories(df, cursor, con, 200, 300)
print("3rd 100 done")

    You can't see it but it took 12 minutes with no change, so lets try something else

In [14]:
def get_category_subcategory(df, index):
    num = df.loc[index, 'Ser']
    name = df.loc[index, 'combined']
    return name, num

def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; j = 0
    
    for i in range(start, end):
        name, num = get_category_subcategory(df, i); num = str(num)

        try:
            cursor.execute("""INSERT INTO "Categories" (category_id, category) VALUES (%s, %s)""", (num, name))
        except psycopg2.errors.UniqueViolation as e:
            listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
        
        if(j==10):
            con.commit(); j = 0 # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions)); return listOfCollisions

In [16]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

collisions = process_categories(df, cursor, con, 0, 100)
print("1st 100 done")

0
1st 100 done


    1st 100 in < 21 seconds - time to get cautiously optimistic

In [17]:
collisions = process_categories(df, cursor, con, 100, 200)
print("2nd 100 done")
collisions = process_categories(df, cursor, con, 200, 300)
print("3rd 100 done")

1
2nd 100 done
0
3rd 100 done


    Looking good, lets build something bigger

    By channeling the for loop, lets try the 1st 1000,
    Which by prospect of 100 entries in under 20 seconds (after pre processing), will take 200s or 3 min 20 s

In [25]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

for i in range(10): # first 10 hundreds
    collisions = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    print("100 number ", (i+1) ," done")
    # print((100*i), ((100*i)+100))

0
100 number  1  done
1
100 number  2  done
0
100 number  3  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

1
100 number  4  done
0
100 number  5  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

1
100 number  6  done
0
100 number  7  done
Error occurred: current transaction is aborted, commands ignored until

    Hm - a number of hiccups, but we have precisely 1000 entries, the last 10 being

| Index | Item                           |
|-------|--------------------------------|
| 990   | Laboratory Bottle              |
| 991   | Laboratory Centrifuge          |
| 992   | Laboratory Clamp               |
| 993   | Laboratory Cylinder            |
| 994   | Laboratory Heating Equipments  |
| 995   | Laboratory Refrigeration Equipments |
| 996   | Laboratory Thermostatic Devices |
| 997   | Other Lab Supplies             |
| 998   | Petri Dish                     |
| 999   | Pipette                        |
| 1000  | Test Tube                      |

now if we run the last 10 in Python:


In [27]:
print(df.iloc[989:1000])

      Ser                             combined
989   990                    Laboratory Bottle
990   991                Laboratory Centrifuge
991   992                     Laboratory Clamp
992   993                  Laboratory Cylinder
993   994        Laboratory Heating Equipments
994   995  Laboratory Refrigeration Equipments
995   996      Laboratory Thermostatic Devices
996   997                   Other Lab Supplies
997   998                           Petri Dish
998   999                              Pipette
999  1000                            Test Tube


    **Victory sound effect**

    Could it be more smooth? of course - nobody likes error messages, but it works, lets try committing all before we begin the cleanup phase

    So therefore: the final step now is to use this to commit our whole data to the DB

In [29]:
# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

for i in range(46): # first 46 hundreds - should do it, but 15 mins seems like an issue - but lets buckle up none the less
    collisions = process_categories(df, cursor, con, (100*i), ((100*i)+100))
    print("100 number ", (i+1) ," done")
    # print((100*i), ((100*i)+100))

0
100 number  1  done
1
100 number  2  done
0
100 number  3  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

1
100 number  4  done
0
100 number  5  done
Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

Error occurred: current transaction is aborted, commands ignored until end of transaction block

1
100 number  6  done
0
100 number  7  done
Error occurred: current transaction is aborted, commands ignored until

KeyError: 4596

In [78]:
cursor.close()
con.close()

In [57]:
# cursor.execute("""select * from "Categories" c""")
# print(cursor.fetchall())

[(3, 'food2'), (4, 'food'), (6, 'food9'), (8, 'food99'), (22, 'food999'), (398, 'Agriculture'), (399, 'Agricultural Equipment'), (400, 'Agricultural Greenhouses'), (401, 'Aquaculture Equipment'), (402, 'Aquaculture Trap'), (403, 'Fishing Float'), (404, 'Fishing Net'), (405, 'Fishing Rope'), (406, 'Other Aquaculture Equipment'), (407, 'Ear Tag')]


In [18]:
# print(df.iloc[300:351]) # I thought the issue was in the dataset, this piece of code was to inspect

In [ ]:
# Ser = []; Cat = []; SCat = []; SSCat = []; SSSCat = []

# for i in dfp:
#     Ser.append(dfp['Ser'])
#     Cat.append(dfp['Category'])
#     SCat.append(dfp['SubCategory'])
#     SSCat.append(dfp['SubSubCategory'])
#     SSSCat.append(dfp['SubSubSubCategory'])

# Ser; # Cat; # SCat; # SSCat; # SSSCat
